# Setup

In [ ]:
import os

def is_running_on_colab():
    return 'COLAB_GPU' in os.environ

if is_running_on_colab():
    "!git clone -b feature/addUi https://github.com/chisakiShinichirouToshiyuki/commentSupporter.git\n",


# OSS
from concurrent.futures import ProcessPoolExecutor
from IPython.display import display
from threading import Thread
from typing import Dict
from typing import List
from typing import TYPE_CHECKING
from typing import TypedDict
import getpass
import ipywidgets as widgets
import json


# 独自モジュール
if TYPE_CHECKING or not is_running_on_colab():
    from my_package.data_formatter import DataFormatter
    from my_package.live_chat_downloader import LiveChatDownloader
    from my_package.analyzer_ui import AnalyzerUi
    from my_package.gpt_handler import チャットGPTハンドラークラス
else:
    from commentSupporter.src.my_package.data_formatter import DataFormatter
    from commentSupporter.src.my_package.live_chat_downloader import LiveChatDownloader
    from commentSupporter.src.my_package.analyzer_ui import AnalyzerUi
    from commentSupporter.src.my_package.gpt_handler import チャットGPTハンドラークラス


In [ ]:
youtube_live_url = input("YoutubeのURLを入力してください: ")
api_key = getpass.getpass("chatGPTを利用する方は、API keyを自己責任で入力してください。: ")
split_url = youtube_live_url.split('https://www.youtube.com/watch?v=')
assert split_url[0] == ''
live_id:str = split_url[1]

# Promptの設定

In [ ]:
チャットGPTハンドラー:チャットGPTハンドラークラス = チャットGPTハンドラークラス(api_key, "あなたはコメント分析の専門家です")
チャットGPTハンドラー.ユーザーメッセージの追加(
    """
        次の一連のコメントから、コメント主の特徴を一言で分析してください。
        「昨日はラーメン食べた」
        「今日はパスタ食べた」
        「明日はうどん食う」
    """)
チャットGPTハンドラー.chatGPT返信の追加("麺好き")
チャットGPTハンドラー.ユーザーメッセージの追加(
    """
        素晴らしい。
        次の一連のコメントから、コメント主の特徴を一言で分析してください。
        「昨日カレー食べた」
        「今日はカレーグラタン食べた」
        「明日はカレーうどん食う」
    """)
チャットGPTハンドラー.chatGPT返信の追加("カレー好き")
チャットGPTハンドラー.ユーザーメッセージの追加(
    """
        素晴らしい。次の一連のコメントから、コメント主の特徴を一言で分析してください。
    """)

# 中略

In [ ]:
def main(live_id: str):
    AnalyzerUi(チャットGPTハンドラー, live_id)

    # プロセスを作成
    executor = ProcessPoolExecutor(max_workers=2)

    # プロセスを開始（別スレッドで実行）
    data_formatter = DataFormatter(live_id, チャットGPTハンドラー)
    live_chat_downloader = LiveChatDownloader(live_id)

    t1 = Thread(target=lambda: executor.submit(data_formatter.replace_watch_comment))
    t1.start()

    t2 = Thread(target=lambda: executor.submit(live_chat_downloader.get_chat))
    t2.start()


# 下記に、チャットとその分析結果が表示されます
## 初回のチャットが分析されるまで、数十秒かかる場合があります

In [ ]:
main(live_id)